# Run ground truth examples and get expected results


## Setup

### Notebook Pre-Req

You can upload this notebook into a Jupyter environment configured to use Neptune Workbench. I tested this on an Amazon Sagemaker notebook running Python 3.10.x. 

### Python Pre-Req

I tested this on an Amazon Sagemaker notebook running Python 3.10.x. You need Python 3.9 or higher.

### Neptune Pre-Req

Your Neptune cluster must run engine version 1.2.x or higher

## Open ground truth file and run the queries against Neptune


In [ ]:
%%bash
#cp -r expected expected1
rm -rf expected
mkdir -p expected

In [ ]:
import yaml
import boto3
import utilities as u
from pathlib import Path

# get ground truth and prefixes
resources = Path.cwd() / "resources"
pfx = (Path.cwd() / "resources" / "prefixes.txt").read_text()
ground_truth = yaml.safe_load((resources / "ground-truth.yaml").read_text())
session=boto3.Session()

def form_sparql(q):
    return f"""
    {pfx}

    {q['SPARQL']}

    LIMIT 20
""".strip()
    

def run_one_test(index):

    q = ground_truth[index]
    nlq=q['question']
    sparql=form_sparql(q)

    try:
        res=u.execute_sparql(sparql, session) 
        u.write_sparql_res("expected", str(index), nlq, q['SPARQL'], sparql, res, "")
    except Exception as e:
        print(f"Error on {index}")
        print("Exception: {}".format(type(e).__name__))
        print("Exception message: {}".format(e))
        error_msg="Exception message: {}".format(e).replace("\n", " ")
        u.write_sparql_res("expected", str(index), nlq, q['SPARQL'], sparql, {}, error_msg)
        print(error_msg)

def run_tests():
    for index, q in enumerate(ground_truth):
        run_one_test(index)

def run_one_up_test(index):

    q = ground_truth[index]
    nlq=q['question']
    sparql=form_sparql(q)

    try:
        res=u.execute_sparql_uniprotref(sparql)
        u.write_sparql_res("up", str(index), nlq, q['SPARQL'], sparql, res, "")
    except Exception as e:
        print(f"Error on {index}")
        print("Exception: {}".format(type(e).__name__))
        print("Exception message: {}".format(e))
        error_msg="Exception message: {}".format(e).replace("\n", " ")
        u.write_sparql_res("up", str(index), nlq, q['SPARQL'], sparql, {}, error_msg)
        print(error_msg)

def run_up_tests():
    for index, q in enumerate(ground_truth):
        run_one_up_test(index)


In [ ]:
%%bash
rm -rf up
mkdir -p up

In [ ]:
run_up_tests()

In [ ]:
run_tests()

In [ ]:
run_one_up_test(34)

In [ ]:
!rm -rf expected/.ipynb_checkpoints up/.ipynb_checkpoints

In [ ]:
u.make_report("expected")
u.make_report("up")
